In [2]:
import numpy as np
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import os

In [20]:
df = pd.read_csv("C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Datos/Datos2011Limipios.csv")
feature_columns = ['cole_depto_ubicacion', 'cole_jornada', 
                      'cole_naturaleza', 'fami_educacionmadre', 'fami_educacionpadre', 
                      'fami_estratovivienda']

In [ ]:
def transform_new_data(new_data, encoders, reference_columns):
    """
    Transforma nuevos datos con los mismos encoders y columnas que el entrenamiento.
    """
    # Verificar que new_data sea un DataFrame
    if not isinstance(new_data, pd.DataFrame):
        raise ValueError("new_data debe ser un DataFrame de pandas")
    
    new_data_encoded = new_data.copy()
    
    # Identificar columnas binarias (que tienen encoder) y multinivel
    binary_columns = [col for col in encoders.keys() if not col.endswith('_emergency')]
    
    # Limpiar y preparar datos
    for col in new_data_encoded.columns:
        # Verificar el dtype de la columna individual
        if pd.api.types.is_object_dtype(new_data_encoded[col]):
            new_data_encoded[col] = new_data_encoded[col].fillna('unknown').astype(str)

    # Aplicar LabelEncoder a columnas binarias
    for col in binary_columns:
        if col in new_data_encoded.columns:
            try:
                # Manejar valores no vistos durante el entrenamiento
                unique_values = set(new_data_encoded[col].unique())
                encoder_classes = set(encoders[col].classes_)
                
                # Si hay valores nuevos, mapearlos al primer valor conocido
                if not unique_values.issubset(encoder_classes):
                    unknown_values = unique_values - encoder_classes
                    for unknown_val in unknown_values:
                        new_data_encoded[col] = new_data_encoded[col].replace(
                            unknown_val, encoders[col].classes_[0]
                        )
                
                new_data_encoded[col] = encoders[col].transform(new_data_encoded[col])
            except Exception as e:
                print(f"Error en columna {col}: {e}")
                new_data_encoded[col] = 0  # Valor por defecto

    # Identificar columnas multinivel de los datos de referencia
    dummy_columns = [col for col in reference_columns 
                    if any(col.startswith(f"{bc}_") for bc in binary_columns) == False]
    
    # Crear dummies para columnas multinivel
    multi_columns_in_data = []
    for ref_col in dummy_columns:
        # Extraer el nombre base de la columna dummy
        for original_col in new_data_encoded.columns:
            if ref_col.startswith(f"{original_col}_") and original_col not in binary_columns:
                if original_col not in multi_columns_in_data:
                    multi_columns_in_data.append(original_col)
    
    if multi_columns_in_data:
        # Crear dummies
        df_multi = new_data_encoded[multi_columns_in_data].copy()
        
        # Asegurar que los valores sean strings
        for col in multi_columns_in_data:
            df_multi[col] = df_multi[col].astype(str)
        
        # Prefijos personalizados (verificando longitud)
        if len(multi_columns_in_data) == len(df_multi.columns):
             df_dummies = pd.get_dummies(df_multi, prefix=multi_columns_in_data, drop_first=True, dtype=int)
        else:
             df_dummies = pd.get_dummies(df_multi, prefix=df_multi.columns, drop_first=True, dtype=int)
        
        # Eliminar columnas originales y agregar dummies
        new_data_encoded = new_data_encoded.drop(columns=multi_columns_in_data, errors='ignore')
        new_data_encoded = pd.concat([new_data_encoded, df_dummies], axis=1)
    
    # Asegurar que tenga todas las columnas de referencia
    for col in reference_columns:
        if col not in new_data_encoded.columns:
            new_data_encoded[col] = 0

    # Reindexar con las columnas de referencia
    new_data_encoded = new_data_encoded.reindex(columns=reference_columns, fill_value=0)
    
    # Asegurar que todas las columnas sean numéricas
    for col in new_data_encoded.columns:
        if pd.api.types.is_object_dtype(new_data_encoded[col]):
            try:
                new_data_encoded[col] = pd.to_numeric(new_data_encoded[col], errors='coerce')
                new_data_encoded[col] = new_data_encoded[col].fillna(0)
            except:
                # Como último recurso, convertir usando un encoder temporal
                le_temp = LabelEncoder()
                new_data_encoded[col] = le_temp.fit_transform(new_data_encoded[col].astype(str))
    
    # Convertir todas las columnas a float32 para consistencia
    for col in new_data_encoded.columns:
        new_data_encoded[col] = new_data_encoded[col].astype('float32')

    return new_data_encoded

In [33]:
def load_best_model_and_predict( modelo, columns_info, new_data_path=None, new_data_df=None):
    """
    Carga el mejor modelo y hace predicciones en nuevos datos
    """
    # Cargar modelo y información de columnas
    model = tf.keras.models.load_model(modelo)
    
    with open(columns_info, "rb") as f:
        columns_info = pickle.load(f)
    
    if new_data_path:
        new_data = pd.read_csv(new_data_path)
    elif new_data_df is not None:
        new_data = new_data_df.copy()
    else:
        # Usar datos de test como ejemplo
        new_data = df[feature_columns].iloc[15:16]  # Primera fila como ejemplo
    
    # Procesar nuevos datos
    new_data_processed = transform_new_data(
        new_data[columns_info['categorical_columns'] + ['fami_estratovivienda']], 
        columns_info['encoders'], 
        columns_info['feature_columns']
    )
    
    # Hacer predicciones
    predictions_prob = model.predict(new_data_processed)
    predictions_binary = (predictions_prob > 0.5).astype(int)
    
    # Convertir a etiquetas legibles
    predictions_labels = ['exitoso' if pred == 1 else 'no_exitoso' for pred in predictions_binary.flatten()]
    
    return {
        'probabilidades': predictions_prob.flatten(),
        'predicciones_binarias': predictions_binary.flatten(),
        'predicciones_etiquetas': predictions_labels,
        'datos_procesados': new_data_processed
    }

In [35]:
# Ejemplo de uso de la función de predicción
print("EJEMPLO DE PREDICCIÓN")

resultados = load_best_model_and_predict(modelo="C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Modelamiento/ejemplo.keras",
                                              columns_info="C:/Users/sofia/Downloads/Repositorios analítica/analítica/proyecto2/Modelamiento/columns_info_ej.pkl")
print("Predicciones en datos de ejemplo:")
for i, (prob, etiqueta) in enumerate(zip(resultados['probabilidades'], resultados['predicciones_etiquetas'])):
        print(f"  Ejemplo {i+1}: {etiqueta} (probabilidad: {prob:.4f})")

EJEMPLO DE PREDICCIÓN
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicciones en datos de ejemplo:
  Ejemplo 1: no_exitoso (probabilidad: 0.0000)
